# 导入库

In [4]:
import pandas as pd
import numpy as np

# 读取数据集

In [3]:
data=pd.read_csv("top1000bigrams.csv")
print(data)

                                             gram  counts
0                               covid 19 pandemic     718
1                                covid 19 vaccine     412
2                          without taking vaccine     371
3                                  covid 19 cases     360
4                               covid covid covid     294
..                                            ...     ...
995                    medidas bioseguridad covid      21
996              doses tds elespoderiamestarvivos      21
997  tds elespoderiamestarvivos vacinaparatodosjá      21
998                            19 casos positivos      21
999                           smart thank serving      21

[1000 rows x 2 columns]


# COVID19图表


In [2]:
#导入相关模块
import time
import json
import requests
from datetime import datetime
import pandas as pd
import numpy as np
import pyecharts.options as opts
from pyecharts.charts import Map
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
CurrentConfig.ONLINE_HOST


# 定义抓取数据函数
def Domestic():
    url = 'https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=statisGradeCityDetail,diseaseh5Shelf'
    response = requests.get(url=url).text
    data = json.loads(response)['data']['diseaseh5Shelf']
    return data

def Oversea():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_foreign'
    reponse = requests.get(url=url).json()
    data = json.loads(reponse['data'])
    return data

domestic = Domestic()
oversea = Oversea()




'''
print(domestic.keys())
print(oversea.keys())
'''




# 提取各地区数据明细
areaTree = domestic['areaTree']
# 查看并分析具体数据
areaTree

# 提取国外地区数据明细
foreignList = oversea['foreignList']
# 查看并分析具体数据
foreignList





china_data = areaTree[0]['children'] 
china_list = []
for a in range(len(china_data)):
    province = china_data[a]['name']  
    confirm = china_data[a]['total']['confirm'] 
    heal = china_data[a]['total']['heal']  
    dead = china_data[a]['total']['dead']  
    nowConfirm = confirm - heal - dead 
    china_dict = {} 
    china_dict['province'] = province  
    china_dict['nowConfirm'] = nowConfirm 
    china_list.append(china_dict) 

china_data = pd.DataFrame(china_list) 
china_data.to_excel("国内疫情.xlsx", index=False) #存储为EXCEL文件
china_data.head()




world_data = foreignList  
world_list = []  

for a in range(len(world_data)):
    # 提取数据
    country = world_data[a]['name']
    nowConfirm = world_data[a]['nowConfirm']  
    confirm = world_data[a]['confirm']
    dead = world_data[a]['dead']  
    heal = world_data[a]['heal'] 
    # 存放数据
    world_dict = {}
    world_dict['country'] = country
    world_dict['nowConfirm'] = nowConfirm
    world_dict['confirm'] = confirm
    world_dict['dead'] = dead
    world_dict['heal'] = heal
    world_list.append(world_dict)

world_data = pd.DataFrame(world_list)
world_data.to_excel("国外疫情.xlsx", index=False)
world_data.head()




confirm = areaTree[0]['total']['confirm']   # 提取中国累计确诊数据
heal = areaTree[0]['total']['heal']         # 提取中国累计治愈数据
dead = areaTree[0]['total']['dead']         # 提取中国累计死亡数据
nowConfirm = confirm - heal - dead          # 计算中国现有确诊数量

world_data = world_data.append(
    {
        'country': "中国",
        'nowConfirm': nowConfirm,
        'confirm': confirm,
        'heal': heal,
        'dead': dead
    },
    ignore_index=True)

province=list(china_data["province"])
nowConfirm=list(china_data["nowConfirm"])
data=[]
i=0
while i< len(province):
    data.append((province[i],nowConfirm[i]))
    i+=1
# print(data)
m = Map()
m.add("",data,
      maptype="china",
      is_map_symbol_show=False)
m.set_global_opts(
    title_opts=opts.TitleOpts(title="COVID-19中国现有地区现有确诊人数地图"),
    visualmap_opts=opts.VisualMapOpts(
        is_piecewise=True,
        pieces=[
            {
                "min": 5000,
                "label": '>5000',
                "color": "#893448"
            },  # 不指定 max，表示 max 为无限大
            {
                "min": 1000,
                "max": 4999,
                "label": '1000-4999',
                "color": "#ff585e"
            },
            {
                "min": 500,
                "max": 999,
                "label": '500-1000',
                "color": "#fb8146"
            },
            {
                "min": 101,
                "max": 499,
                "label": '101-499',
                "color": "#ffA500"
            },
            {
                "min": 10,
                "max": 100,
                "label": '10-100',
                "color": "#ffb248"
            },
            {
                "min": 1,
                "max": 9,
                "label": '1-9',
                "color": "#fff2d1"
            },
            {
                "max": 1,
                "label": '0',
                "color": "#ffffff"
            }
        ]))
m.render()

/tmp/ipykernel_25936/1257542538.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_data = world_data.append(


'/home/dky/Github/Mine/python-practice/day07/covid-19/render.html'